In [ ]:
from IPython.display import YouTubeVideo
import cv2
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import youtube_dl as yt
%matplotlib inline
%load_ext autoreload
%autoreload

In [ ]:
sample_video_id = 'C5IkONJ-aTY'

In [ ]:
# Download sample driving video
ydl_opts = {"outtmpl" : "data/video/%(id)s.%(ext)s", 
            "writedescription" : True,
            "writeannotations" : True,
            "writethumbnail" : True,
            "format" : "mp4"
           }

for video in [sample_video_id]:
    if not os.path.isfile("data/video/" + video + ".mp4"):
        yt.YoutubeDL(ydl_opts).download(["http://www.youtube.com/watch?v=" + video])

In [ ]:
cap = cv2.VideoCapture("data/video/" + sample_video_id + ".mp4")
print("The total number of frames " + str(int(cap.get(7))))

In [ ]:
cap.set(cv2.CAP_PROP_POS_MSEC, 49000)
bool_flag, test_frame = video_reader.read()
plt.imshow(test_frame[:, :, [2, 1, 0]])
plt.show()

In [ ]:
def get_flow_map(cap, msec1, msec2):

    # params for ShiTomasi corner detection
    feature_params = dict( maxCorners = 100,
                           qualityLevel = 0.4,
                           minDistance = 7,
                           blockSize = 7 )
    # Parameters for lucas kanade optical flow
    lk_params = dict( winSize  = (15,15),
                      maxLevel = 2,
                      criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    # Create some random colors
    color = np.random.randint(0,255,(100,3))
    # Take first frame and find corners in it
    cap.set(cv2.CAP_PROP_POS_MSEC, msec1)
    _, old_frame = cap.read()
    old_frame = old_frame[350:,:]
    
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
    p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)
    # Create a mask image for drawing purposes
    mask = np.zeros_like(old_frame)
    
    cap.set(cv2.CAP_PROP_POS_MSEC, msec2)
    _, frame = cap.read()
    frame = frame[350:,:]
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # calculate optical flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
    # Select good points
    good_new = p1[st==1]
    good_old = p0[st==1]
    
    # draw the tracks
    vectors = []
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        vectors.append([math.pow(c-a, 2) + math.pow(d-b, 2), a,b,c,d])
    
    vectors = np.array(vectors, dtype=np.int32)
    length_threshold = np.percentile(vectors[:,0], q=75)
    
    for i in range(len(vectors)):
        length,a,b,c,d = vectors[i,:]
        if length >= length_threshold:
            mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
            frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)

    plt.imshow(old_frame[:, :, [2, 1, 0]])
    plt.show()
    plt.imshow(frame[:, :, [2, 1, 0]])
    plt.show()
    plt.imshow(img[:, :, [2, 1, 0]])
    plt.show()
    
    vectors = vectors[vectors[:,0] > length_threshold]
    return np.mean(vectors[:,3] - vectors[:,1]), np.mean(vectors[:,4] - vectors[:,2])
    
avg_vector = get_flow_map(cap, 75000, 76000)
print(avg_vector)

In [ ]:
# Playing with optical flow
def get_flow_map(cap, msec1, msec2):
    cap.set(cv2.CAP_PROP_POS_MSEC, msec1)
    _, frame1 = cap.read()
    cap.set(cv2.CAP_PROP_POS_MSEC, msec2)
    _, frame2 = cap.read()
    
    frame1_map = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    frame2_map = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    
    hsv = np.zeros_like(frame1)
    hsv[...,1] = 255

    flow = cv2.calcOpticalFlowFarneback(frame1_map, frame2_map, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    plt.imshow(frame1[:, :, [2, 1, 0]])
    plt.show()
    plt.imshow(frame2[:, :, [2, 1, 0]])
    plt.show()
#     plt.imshow(hsv[:, :, [2, 1, 0]])
#     plt.show()
#     plt.imshow(bgr[:, :, [2, 1, 0]])
#     plt.show()
#     k = cv2.waitKey(30) & 0xff
#     if k == 27:
#         break
#     elif k == ord('s'):
#         cv2.imwrite('opticalfb.png',frame2)
#         cv2.imwrite('opticalhsv.png',bgr)
#     prvs = next

    return np.ma.masked_invalid(mag).mean(), np.ma.masked_invalid(ang * mag).mean()
#     return np.ma.masked_invalid(mag).mean(), np.ma.masked_invalid(ang).mean()

mag, ang = get_flow_map(cap, 75000, 76000)
print(mag, ang)

In [ ]:
cap.release()